In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
dataset = pd.read_excel("../../data/raw/File_10_-_IoD2019_Local_Authority_District_Summaries__lower-tier__.xlsx", sheet_name="IMD")
filtered_panel = pd.read_csv(r'../../data/processed/final_panel_data.csv')

In [3]:
dataset = dataset.rename(columns={'Local Authority District name (2019)': 'local_authority'})
dataset['local_authority'] = dataset['local_authority'].str.lower().str.strip()
dataset = dataset.rename(columns={'IMD - Average score ': 'imd_score'})
dataset['local_authority'] = (
    dataset['local_authority']
    .str.lower()
    .str.replace('-', ' ', regex=False)
    .str.replace(',', ' ', regex=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
dataset['local_authority'] = dataset['local_authority'].replace({
    #'barrow in furness': 'westmorland and furness',
    #'north somerset': 'somerset',
    'kingston upon hull city of': 'kingston upon hull',
    'herefordshire county of': 'herefordshire',
    'bristol city of': 'bristol',
    #'southend on sea': 'southend-on-sea',
    'st. helens': 'st helens', 
})
unified_map = {
    'buckinghamshire': ['aylesbury vale', 'chiltern', 'south bucks', 'wycombe','south buckinghamshire'],
    'dorset': ['weymouth and portland', 'west dorset', 'north dorset', 'east dorset', 'purbeck', 'christchurch'],
    'somerset': ['taunton deane', 'west somerset', 'mendip', 'sedgemoor', 'south somerset', 'somerset council','somerset west and taunton'],
    'cumberland': ['allerdale', 'carlisle', 'copeland', 'cumberland council'],
    'westmorland and furness': ['barrow in furness', 'barrow-in-furness', 'eden', 'south lakeland'],
    'north yorkshire': ['craven', 'hambleton', 'harrogate', 'richmondshire', 'ryedale', 'scarborough', 'selby', 'north yorkshire council'],
    'bournemouth christchurch and poole': ['bournemouth', 'christchurch', 'poole'],
    'west suffolk': ['forest heath', 'st edmundsbury'],
    'east suffolk': ['suffolk coastal', 'waveney'],
    'bath and north east somerset': ['bath and ne somerset'],
    'southend-on-sea': ['southend on sea'],
    'leicester': ['leicester city'],
    'medway': ['medway towns'],
    'derby': ['derby city'],
    'folkestone and hythe': ['shepway'],
    'county durham': ['durham'],
    "king's lynn and west norfolk": ['kings lynn and west norfolk'],
    'north northamptonshire': ['wellingborough', 'east northamptonshire', 'kettering', 'corby'],
    'west northamptonshire': ['northampton', 'south northamptonshire', 'daventry'],
}
# Create reverse mapping: value → key
reverse_map = {
    alt_name: unified for unified, alts in unified_map.items() for alt_name in alts
}

# Replace values in dataset['local_authority'] using the reverse_map
dataset['local_authority'] = dataset['local_authority'].replace(reverse_map)

dataset = (
    dataset.groupby('local_authority', as_index=False)['imd_score']
    .mean()
)


In [4]:
filtered_panel = filtered_panel.merge(
    dataset[['local_authority', 'imd_score']],
    on='local_authority',
    how='left'
)

In [5]:
filtered_panel.loc[filtered_panel['imd_score'].isna(), 'local_authority'].unique()

array([], dtype=object)

In [6]:
model = smf.ols(
    formula=(
        'removals ~ '
        'value + value_lag1 + value_lag2 + value_lag3 + '
        'imd_score + C(financial_year) + C(size_category) + '
        'value:C(size_category) + '
        'value_lag1:C(size_category) + '
        'value_lag2:C(size_category) + '
        'value_lag3:C(size_category)'
    ),
    data=filtered_panel
).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               removals   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     264.9
Date:                Fri, 25 Jul 2025   Prob (F-statistic):               0.00
Time:                        15:24:00   Log-Likelihood:                -15969.
No. Observations:                5322   AIC:                         3.198e+04
Df Residuals:                    5301   BIC:                         3.212e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------